In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [58]:
df_gowalla = pd.read_csv('..\data\loc-gowalla_totalCheckins.txt\Gowalla_totalCheckins.txt', sep='\t', header=None)
df_gowalla.columns = ['userid','timestamp','latitude','longitude','spotid']
df_gowalla.head()

,userid,timestamp,latitude,longitude,spotid
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [59]:
lon_min, lat_min, lon_max, lat_max = -74.2589, 40.4774, -73.7004, 40.9176
nyc_events = df_gowalla[(df_gowalla['longitude']>lon_min) & 
           (df_gowalla['longitude']<lon_max) & 
           (df_gowalla['latitude']>lat_min) & 
           (df_gowalla['latitude']<lat_max)]
nyc_events.head()

,userid,timestamp,latitude,longitude,spotid
10,0,2010-10-12T00:21:28Z,40.643885,-73.782806,23261
11,0,2010-10-11T20:21:20Z,40.741374,-73.988105,16907
12,0,2010-10-11T20:20:42Z,40.741388,-73.989455,12973
13,0,2010-10-11T00:06:30Z,40.724910,-73.994621,341255
14,0,2010-10-10T22:00:37Z,40.729768,-73.998535,260957


In [60]:
venues = pd.read_csv('..\data\spots.txt\spots.txt', sep='\t', header=0)
venues.head()

,spotid,spotname,geo-coordinates
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436)
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413)
2,3612422,Conference House,POINT(-74.2490418254 40.5000644614)
3,3612431,Billop House,POINT(-74.2490418254 40.5000644614)
4,1391499,Biddle House,POINT(-74.2541456223 40.5054829933)


In [61]:
#UTC to Newyork Time Zone Conversion

def parse_datetime(s):
    tzone = tz.timezone("America/New_York") #parse_datetime
    utc = datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')
    return tz.utc.localize(utc).astimezone(tzone)

nyc_events = pd.DataFrame.merge(nyc_events, venues[['spotid','spotname']], on='spotid', how="inner")
nyc_events.head()
# saving the dataframe 
nyc_events.to_csv('..\\dataframes\\nyc_events_with_name.csv') 

In [62]:
#movies = pd.read_csv("data\\movies.csv",encoding="Latin1")
Ratings = pd.read_csv("..\\data\\ratings.csv")
#Tags = pd.read_csv("data\\tags.csv",encoding="Latin1")


In [63]:
Ratings['spotid'].describe()

count    264505.000000
mean      11917.340326
std       21615.562394
min           1.000000
25%        1260.000000
50%        2949.000000
75%        6537.000000
max      112852.000000
Name: spotid, dtype: float64

In [64]:
Ratings['spotid'].nunique()

2500

In [65]:
ratings_map = {}
num = 0
for i in np.unique(Ratings['spotid']):
    ratings_map[i] = num
    Ratings['spotid'] = Ratings['spotid'].replace(i,num)
    num = num+1

In [66]:
Ratings['spotid'].describe()

count    264505.000000
mean       1321.221402
std         746.922097
min           0.000000
25%         644.000000
50%        1350.000000
75%        2011.000000
max        2499.000000
Name: spotid, dtype: float64

In [67]:
# saving the dataframe 
Ratings.to_csv('..\\dataframes\\Ratings.csv') 

In [68]:
Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userId,spotid,rating_x,timestamp,rating_y,adg_rating
0,12882,0,4.0,1147195252,4.061321,-0.061321
1,12882,29,3.5,1147195307,4.061321,-0.561321
2,12882,40,5.0,1147195343,4.061321,0.938679
3,12882,42,5.0,1147185499,4.061321,0.938679
4,12882,76,4.5,1147195239,4.061321,0.438679


In [69]:
# saving the dataframe 
Rating_avg.to_csv('..\\dataframes\\Rating_avg.csv') 

In [70]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='spotid')
check.head()

spotid,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,4.5,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,2.5,3.0,4.5,4.0,NaN,NaN,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,NaN,3.5,3.5,NaN,NaN,NaN,4.0,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN


In [71]:
from sklearn.metrics.pairwise import cosine_similarity
final=pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='spotid')

In [72]:
final.shape

(862, 2500)

In [73]:
final.head()

spotid,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,NaN,NaN,NaN,NaN,NaN,-1.329457,NaN,-0.829457,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.314526,NaN,NaN,NaN,NaN,1.314526,NaN,NaN,0.314526,0.314526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.705596,0.205596,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,...,-1.294404,-0.794404,0.705596,0.205596,NaN,NaN,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.39808,NaN,-0.39808,-0.398080,NaN,NaN,NaN,0.101920,...,NaN,NaN,-0.398080,NaN,NaN,NaN,NaN,0.601920,NaN,NaN


In [74]:
final_location = final.fillna(final.mean(axis=0))

In [75]:
final_location.head()

spotid,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-1.329457,-0.690175,-0.829457,-0.094277,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
320,0.200220,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-0.203889,-0.690175,-0.150642,-0.094277,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
359,1.314526,-0.436518,-0.468109,-0.770223,-0.615331,1.314526,-0.203889,-0.690175,0.314526,0.314526,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
370,0.705596,0.205596,-0.468109,-0.770223,-0.615331,1.205596,-0.203889,-0.690175,-0.150642,-0.094277,...,-1.294404,-0.794404,0.705596,0.205596,0.429868,0.306567,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.398080,-0.770223,-0.398080,-0.398080,-0.203889,-0.690175,-0.150642,0.101920,...,0.105075,0.006629,-0.398080,0.237350,0.429868,0.306567,0.225110,0.601920,0.362468,0.349157


In [76]:
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [77]:
final_user.head()

spotid,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
userId,,,,,,,,,,,,,,,,,,,,,
316,-8.294574e-01,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.329457e+00,-1.445872e-16,-8.294574e-01,-1.445872e-16,...,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16
320,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,...,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17
359,1.314526e+00,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,1.314526e+00,9.809414e-17,9.809414e-17,3.145258e-01,3.145258e-01,...,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17
370,7.055961e-01,2.055961e-01,-1.728814e-16,-1.728814e-16,-1.728814e-16,1.205596e+00,-1.728814e-16,-1.728814e-16,-1.728814e-16,-1.728814e-16,...,-1.294404e+00,-7.944039e-01,7.055961e-01,2.055961e-01,-1.728814e-16,-1.728814e-16,-7.944039e-01,7.055961e-01,-2.944039e-01,-7.944039e-01
910,1.101920e+00,1.019202e-01,-3.980798e-01,-1.994138e-16,-3.980798e-01,-3.980798e-01,-1.994138e-16,-1.994138e-16,-1.994138e-16,1.019202e-01,...,-1.994138e-16,-1.994138e-16,-3.980798e-01,-1.994138e-16,-1.994138e-16,-1.994138e-16,-1.994138e-16,6.019202e-01,-1.994138e-16,-1.994138e-16


In [78]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,...,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,-1.810374e-31,0.067389,...,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,...,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,...,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,...,0.039464,-0.017620,0.020058,-0.004581,0.040866,-0.001438,-0.026082,0.073272,-0.012058,0.007610


In [79]:
# user similarity on replacing NAN by item(movie) avg

cosine = cosine_similarity(final_location)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_location.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.921169,0.665659,0.673486,0.694247,0.894969,0.805780,0.851492,0.945224,0.705491,...,0.827564,0.895641,0.879290,0.916856,0.912146,0.922262,0.587738,0.671783,0.949138,0.740220
320,0.921169,0.000000,0.687225,0.691158,0.699527,0.916020,0.816931,0.874283,0.970234,0.724147,...,0.861798,0.909376,0.907009,0.938964,0.929049,0.943265,0.612746,0.695382,0.973853,0.768459
359,0.665659,0.687225,0.000000,0.534369,0.523475,0.655225,0.602806,0.629143,0.705042,0.542504,...,0.621820,0.654320,0.655839,0.679696,0.683900,0.686193,0.418283,0.489595,0.707370,0.534065
370,0.673486,0.691158,0.534369,0.000000,0.547560,0.671810,0.618456,0.628825,0.712683,0.548592,...,0.636688,0.673489,0.651209,0.688647,0.689265,0.692595,0.405881,0.497332,0.714011,0.546637
910,0.694247,0.699527,0.523475,0.547560,0.000000,0.680701,0.621463,0.634921,0.723574,0.528281,...,0.638257,0.668887,0.677377,0.701964,0.701245,0.705041,0.408456,0.509008,0.725896,0.554105


In [80]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [103]:
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head(10)

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
316,113673,117918,9050,12882,38187,102668,98880,43829,13215,78501,...,88608,120782,74472,53834,88928,42245,58265,89527,49830,63902
320,12288,113673,28159,79846,134627,112948,120729,97163,2945,4931,...,39271,94883,127683,101137,54989,134521,80946,10055,64365,106512
359,102118,96482,102532,50898,2702,60016,23428,120782,57937,42096,...,117258,7723,120729,61305,40768,117918,86768,129498,131620,58346
370,46645,42245,40768,23428,123707,60016,45120,113645,97195,102118,...,5611,20530,2702,38159,359,43354,117144,96482,2988,108195
910,87042,131620,67352,40768,31321,48821,26222,63295,5611,370,...,134521,88738,46645,108195,70201,58265,18115,114601,23428,17039
975,42245,84549,44400,65670,135223,66455,101137,64100,128224,59269,...,16581,132295,45015,134521,12569,12649,137118,68905,47150,114601
1015,98541,89988,52004,11343,125012,115852,18675,108285,29631,52437,...,70905,27838,90145,89321,23171,1588,120729,42660,22352,23068
1387,101952,102668,45811,28340,11337,36727,96370,92409,50898,38187,...,93710,44400,138200,67352,76164,52004,111841,46645,26172,36665
1447,104397,138176,83360,27484,42015,54643,44400,120320,93710,54989,...,70905,90216,85824,10514,21860,51255,17278,120572,67352,59527


In [82]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
316,138176,100240,96936,51460,88932,1447,104732,125012,5268,121403,...,121987,72633,21401,114335,22338,118304,124981,93203,81435,94333
320,138176,96936,121403,1447,51460,125012,88932,42944,5268,104529,...,121987,102549,118304,86309,94333,124981,93203,80585,136037,22338
359,138176,1447,5268,96936,100240,21401,88932,13927,104732,72633,...,12930,121987,114335,125012,51460,118304,57474,27142,80585,22338
370,86309,44194,138176,24802,129869,96936,1447,104529,94333,88932,...,124981,27142,102549,120308,54643,42944,80585,13927,21401,136037
910,96936,107991,138176,27142,51460,125012,88932,100240,72633,129869,...,36624,51255,94333,42944,121403,80585,61755,124981,88455,78908


In [83]:
spot = {}
num = 0
for i in np.unique(nyc_events['spotid']):
    spot[i] = num
    nyc_events['spotid'] = nyc_events['spotid'].replace(i,num)
    num = num+1

In [84]:
nyc_events.head()

,userid,timestamp,latitude,longitude,spotid,spotname
0,0,2010-10-12T00:21:28Z,40.643885,-73.782806,249,JFK John F. Kennedy International
1,0,2010-10-07T15:27:40Z,40.643885,-73.782806,249,JFK John F. Kennedy International
2,5,2010-08-16T03:47:20Z,40.643885,-73.782806,249,JFK John F. Kennedy International
3,8,2010-10-06T20:23:14Z,40.643885,-73.782806,249,JFK John F. Kennedy International
4,15,2010-10-13T04:10:36Z,40.643885,-73.782806,249,JFK John F. Kennedy International


In [85]:
nyc_events.describe()

,userid,latitude,longitude,spotid
count,112390.000000,112390.000000,112390.000000,112390.000000
mean,41725.282792,40.739526,-73.983131,5208.571279
std,47784.747785,0.044375,0.068852,4852.817609
min,0.000000,40.484370,-74.258598,0.000000
25%,4595.000000,40.722671,-73.998459,933.000000
50%,24970.000000,40.743496,-73.986991,3690.000000
75%,66746.000000,40.759771,-73.975262,8672.000000
max,196524.000000,40.917585,-73.740295,17290.000000


In [86]:
nyc_events_clean = nyc_events[nyc_events['spotid'] < 2500]

In [87]:
nyc_events_clean.shape

(46058, 6)

In [88]:
nyc_events_clean.drop(['userid', 'timestamp','latitude','longitude'], axis=1, inplace = True) 
nyc_events_clean.drop_duplicates(subset='spotid', keep="last", inplace = True)

nyc_events_clean.head()

,spotid,spotname
1653,249,JFK John F. Kennedy International
1932,172,Shake Shack
2102,110,Flatiron Building
2245,1630,Market
2258,420,Westville


In [89]:
# saving the dataframe 
nyc_events_clean.to_csv('..\\dataframes\\nyc_events_clean.csv') 

In [90]:
def get_similar_locations(user1, user2):
    common_locations = Rating_avg[Rating_avg.userId == user1].merge(
        Rating_avg[Rating_avg.userId == user2],
        on = "spotid",
        how ="inner"
    )
    return common_locations.merge(nyc_events_clean, on = 'spotid')


In [91]:
a = get_similar_locations(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','spotname']]
a

,rating_x_x,rating_x_y,spotname
0,5.0,5.0,Mercadito
1,5.0,4.5,Clandestino
2,5.0,4.0,Central Parking
3,4.5,4.0,Tompkins Square Park Playground
4,1.5,1.0,Hot Mini Cakes
5,3.5,3.5,Scuderia
6,5.0,3.5,Peace & Love
7,3.0,3.5,NYU Psychology Building
8,4.0,4.0,NYU Torch Club
9,1.0,4.0,Park Slope Ale House


In [92]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_location.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score    

In [93]:
score = User_item_score(320,2498)
print("score (u,i) is",score)

score (u,i) is 4.064080591401218


In [94]:
Rating_avg = Rating_avg.astype({"spotid": str})
Location_user = Rating_avg.groupby(by = 'userId')['spotid'].apply(lambda x:','.join(x))

In [95]:
def User_item_score1(user):
    Locations_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Location_user[Location_user.index.isin(b)]
    l = ','.join(d.values)
    Locations_seen_by_similar_users = l.split(',')
    Locations_under_consideration = list(set(Locations_seen_by_similar_users)-set(list(map(str, Locations_seen_by_user))))
    Locations_under_consideration = list(map(int, Locations_under_consideration))
    score = []
    for item in Locations_under_consideration:
        c = final_location.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'spotid':Locations_under_consideration,'score':score})
    top_30_recommendation = data.sort_values(by='score',ascending=False).head(30)
    Location_Name = top_30_recommendation.merge(nyc_events_clean, how='inner', on='spotid')
    Location_Names = Location_Name.spotname.values.tolist()
    return Location_Names

In [106]:

#user = int(input("Enter the user id to whom you want to recommend : "))
predicted_locs = User_item_score1(320)  
#print("The Recommendations for User Id : "+user)
for i in predicted_locs:
    print(i)


Traffic
The Park
Minetta Tavern
Republic
Green-Wood Cemetery
The Riviera Cafe
Battery Park Playground
Starbucks Coffee
The Grey Dog's Coffee
Venda
Silver Center
Hoboken Terminal
New Victory Theater
Great Lakes
AMC Empire 25
Canal Room
Planet Hollywood New York
64 Morton
Quartino
Café Grumpy
Blue Man Group, Astor Place
Rosetta
Whole Foods Market (Columbus Circle)
Apollo Theater
Rivergate Veterinary
74th Street Generating Station
Weather Up
Myers Of Keswick
Johny's
Dusk


In [107]:
collabDf = pd.DataFrame(columns=['spotid', 'spotname'])

for spotname in predicted_locs:
    spotid=nyc_events[nyc_events['spotname']==spotname]['spotid'].iloc[0]
    collabDf = collabDf.append({'spotid': spotid , 'spotname': spotname}, ignore_index=True)

# saving the dataframe 
collabDf.to_csv('..\\dataframes\\collabDf.csv') 

collabDf[:15]

,spotid,spotname
0,1848,Traffic
1,435,The Park
2,632,Minetta Tavern
3,591,Republic
4,958,Green-Wood Cemetery
5,451,The Riviera Cafe
6,175,Battery Park Playground
7,3446,Starbucks Coffee
8,426,The Grey Dog's Coffee
9,1932,Venda
